# MongoDB Load JSON with python

In [1]:
%matplotlib inline
import pymongo
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from bson import json_util, ObjectId
import pandas as pd
from pandas import DataFrame
from pandas.io.json import json_normalize
import numpy as np
import requests
import json, os
import configparser
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
import warnings
import random
import pprint
from datetime import datetime
random.seed(datetime.now())
warnings.filterwarnings('ignore')

# Make plots larger
plt.rcParams['figure.figsize'] = (10, 6)

In [2]:
config = configparser.ConfigParser()
config.read('./data/mongo.ini')

['./data/mongo.ini']

In [6]:
client = MongoClient(config.get('Mongo', 'host'), int(config.get('Mongo', 'port')))

In [7]:
db=client.tweets

In [8]:
db.collection_names(include_system_collections=False)

[]

In [9]:
try:
    result = db.tweets.drop()
    print ("analytics tweets dropped")
except:
    pass

analytics tweets dropped


In [10]:
def load_json(j):
    p=os.path.join("data/", j)
    print (p)
    with open(p, 'rU') as f:
      data = [json.loads(row) for row in f]
    return data 

In [11]:
tweets_j=load_json('db_tweets.json')
tweets_j[0]

data/db_tweets.json


{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Mar 15 21:41:22 +0000 2015',
 'entities': {'hashtags': [{'indices': [128, 136], 'text': 'Bigdata'},
   {'indices': [137, 140], 'text': 'IoT'},
   {'indices': [139, 140], 'text': 'CIO'}],
  'symbols': [],
  'trends': [],
  'urls': [{'display_url': 'bit.ly/1jvQFcK',
    'expanded_url': 'http://bit.ly/1jvQFcK',
    'indices': [105, 127],
    'url': 'http://t.co/gLf50KtZKx'}],
  'user_mentions': [{'id': 474651213,
    'id_str': '474651213',
    'indices': [3, 15],
    'name': 'Marc Wilczek',
    'screen_name': 'MarcWilczek'}]},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 577223085054558208,
 'id_str': '577223085054558208',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'lang': 'en',
 'place': None,
 'possibly_sensitive': False,
 'retweet_count': 0,
 'retweeted': Fa

In [12]:
print(tweets_j[0]['text'])

RT @MarcWilczek: In 2014, the digital universe equaled 1.7 megabytes a minute for every person on Earth: http://t.co/gLf50KtZKx #Bigdata #I…


In [13]:
def tweet_json(tid,text,created):
    j={
    "tweet_id" : tid,
    "text" : text,
    "created_at" : created}
    return j

In [14]:
# Twitter dates are of the form Sun Mar 15 21:41:54 +0000 2015
datestrings=['Sun Mar 15 21:41:54 +0000 2015','Tue Mar 29 08:11:25 +0000 2011']
from datetime import timedelta
from email.utils import parsedate_tz
from dateutil.parser import parse

def to_datetime(datestring):
    time_tuple = parsedate_tz(datestring.strip())
    dt = datetime(*time_tuple[:6])
    return dt - timedelta(seconds=time_tuple[-1])

ts=to_datetime(datestrings[0])
print (ts.strftime("%Y-%m-%d %H:%M"))
ts=to_datetime(datestrings[1])
print (ts.strftime("%Y-%m-%d %H:%M"))

2015-03-15 21:41
2011-03-29 08:11


In [15]:
cnt=0
for tweet in tweets_j:
    ts=datetime.now()
    try:
        ts=to_datetime(tweet['created_at'])
    except:
        continue     
    try:
        j=tweet_json(tweet['id'],tweet['text'],ts)
        result = db.tweets.insert_one(j)
        cnt+=1
    except:
        pass       
print ("%d tweets inserted."%cnt)

55 tweets inserted.


In [16]:
for tweet in db.tweets.find():
    pprint.pprint(tweet)

{'_id': ObjectId('5ac442b3a313fc06a48193ee'),
 'created_at': datetime.datetime(2015, 3, 15, 21, 41, 22),
 'text': 'RT @MarcWilczek: In 2014, the digital universe equaled 1.7 megabytes '
         'a minute for every person on Earth: http://t.co/gLf50KtZKx #Bigdata '
         '#I…',
 'tweet_id': 577223085054558208}
{'_id': ObjectId('5ac442b3a313fc06a48193ef'),
 'created_at': datetime.datetime(2015, 3, 15, 21, 41, 30),
 'text': 'Tanginang database to',
 'tweet_id': 577223118327926785}
{'_id': ObjectId('5ac442b3a313fc06a48193f0'),
 'created_at': datetime.datetime(2015, 3, 15, 21, 41, 31),
 'text': 'RT @chrisco: The future of farming: robots + big data: '
         'http://t.co/MiEIZmKzEb via @KurzweilAINews #AgTech #AgBots #Robotics '
         '#Automation #Io…',
 'tweet_id': 577223124862697472}
{'_id': ObjectId('5ac442b3a313fc06a48193f1'),
 'created_at': datetime.datetime(2015, 3, 15, 21, 41, 40),
 'text': 'RT @Brian_Singer_: The EMC Federation Joins the OpenStack Foundation '
         'ht

Updated October 3, 2017